In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import annotations

import os
import sys
from pprint import pprint

# import numpy as np
import lamindb as ln
import numpy as np
import pandas as pd
import psutil
import pytest
import scanpy as sc
import tqdm
from lightning.pytorch import LightningDataModule
from torch.utils.data import DataLoader

import scvi
from scvi.data import synthetic_iid

! schema module 'ourprojects' is not installed → no access to its labels & registries (resolve via `pip install ourprojects`)
→ connected lamindb: laminlabs/cellxgene


/home/canergen/.local/share/hatch/env/virtual/scvi-tools/SQ6yRUPo/scvi-tools/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_datamodule' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/canergen/.local/share/hatch/env/virtual/scvi-tools/SQ6yRUPo/scvi-tools/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_source_registry' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


In [41]:
collection.mapped?

Signature:
collection.mapped(
    layers_keys: 'str | list[str] | None' = None,
    obs_keys: 'str | list[str] | None' = None,
    obsm_keys: 'str | list[str] | None' = None,
    obs_filter: 'tuple[str, str | tuple[str, ...]] | None' = None,
    join: "Literal['inner', 'outer'] | None" = 'inner',
    encode_labels: 'bool | list[str]' = True,
    unknown_label: 'str | dict[str, str] | None' = None,
    cache_categories: 'bool' = True,
    parallel: 'bool' = False,
    dtype: 'str | None' = None,
    stream: 'bool' = False,
    is_run_input: 'bool | None' = None,
) -> 'MappedCollection'
Docstring:
Return a map-style dataset.

Returns a `pytorch map-style dataset
<https://pytorch.org/docs/stable/data.html#map-style-datasets>`__ by
virtually concatenating `AnnData` arrays.

If your `AnnData` collection is in the cloud, move them into a local
cache first via :meth:`~lamindb.Collection.cache`.

`__getitem__` of the `MappedCollection` object takes a single integer index
and returns a dictiona

In [46]:
datamodule.label_keys

{'unknown': -1,
 'B cell': 0,
 'B-1a B cell': 1,
 'B-1b B cell': 2,
 'CD14-positive monocyte': 3,
 'CD16-negative, CD56-bright natural killer cell, human': 4,
 'CD16-positive, CD56-dim natural killer cell, human': 5,
 'CD1c-positive myeloid dendritic cell': 6,
 'CD34-positive, CD56-positive, CD117-positive common innate lymphoid precursor, human': 7,
 'CD4-positive, alpha-beta memory T cell': 8,
 'CD8-positive, alpha-beta T cell': 9,
 'IgA plasma cell': 10,
 'IgG plasma cell': 11,
 'Schwann cell': 12,
 'Schwann cell precursor': 13,
 'T cell': 14,
 'T-helper 17 cell': 15,
 'adventitial cell': 16,
 'airway submucosal gland duct basal cell': 17,
 'alveolar macrophage': 18,
 'basal cell': 19,
 'basophil': 20,
 'blood vessel endothelial cell': 21,
 'blood vessel smooth muscle cell': 22,
 'bone marrow cell': 23,
 'bronchial smooth muscle cell': 24,
 'bronchus fibroblast of lung': 25,
 'brush cell': 26,
 'capillary endothelial cell': 27,
 'chondrocyte': 28,
 'ciliated cell': 29,
 'ciliated ep

In [57]:
class MappedCollectionDataModule(LightningDataModule):
    def __init__(
        self,
        collection: ln.Collection,
        batch_key: str | None = None,
        label_key: str | None = None,
        batch_size: int = 128,
        **kwargs
    ):
        self._batch_size = batch_size
        self._batch_key = batch_key
        self._label_key = label_key
        self._parallel = kwargs.pop("parallel", True)
        # here we initialize MappedCollection to use in a pytorch DataLoader
        self._dataset = collection.mapped(
            obs_keys=[self._batch_key, self._label_key],
            parallel=self._parallel,
            unknown_label="unknown",
            **kwargs
        )
        # need by scvi and lightning.pytorch
        self._log_hyperparams = False
        self.allow_zero_length_dataloader_with_multiple_devices = False

    def close(self):
        self._dataset.close()

    def setup(self, stage):
        pass

    def train_dataloader(self):
        return self._create_dataloader(shuffle=True)

    def inference_dataloader(self):
        """Dataloader for inference with `on_before_batch_transfer` applied."""
        dataloader = self._create_dataloader(shuffle=False, batch_size=4096)
        return self._InferenceDataloader(dataloader, self.on_before_batch_transfer)

    def _create_dataloader(self, shuffle, batch_size=None):
        if self._parallel:
            num_workers = psutil.cpu_count() - 1
            worker_init_fn = self._dataset.torch_worker_init_fn
        else:
            num_workers = 0
            worker_init_fn = None
        if batch_size is None:
            batch_size = self._batch_size
        return DataLoader(
            self._dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            worker_init_fn=worker_init_fn,
        )

    @property
    def n_obs(self) -> int:
        return self._dataset.n_obs

    @property
    def var_names(self) -> int:
        return self._dataset.var_joint

    @property
    def n_vars(self) -> int:
        return self._dataset.n_vars

    @property
    def n_batch(self) -> int:
        if self._batch_key is None:
            return 1
        return len(self._dataset.encoders[self._batch_key])

    @property
    def n_labels(self) -> int:
        if self._label_key is None:
            return 1
        return len(self._dataset.encoders[self._label_key])

    @property
    def labels(self) -> np.ndarray:
        return self._dataset[self._label_key]

    @property
    def registry(self) -> dict:
        return{
            'scvi_version': scvi.__version__,
            'model_name': 'SCVI',
            'setup_args': {
                'layer': None,
                'batch_key': self._batch_key,
                'labels_key': self._label_key,
                'size_factor_key': None,
                'categorical_covariate_keys': None,
                'continuous_covariate_keys': None,
            },
            'field_registries': {
                'X': {'data_registry': {'attr_name': 'X', 'attr_key': None},
                    'state_registry': {'n_obs': self.n_obs,
                    'n_vars': self.n_vars,
                    'column_names': self.var_names},
                    'summary_stats': {'n_vars': self.n_vars, 'n_cells': self.n_obs}},
                'batch': {'data_registry': {'attr_name': 'obs',
                    'attr_key': '_scvi_batch'},
                    'state_registry': {'categorical_mapping': self.batch_keys,
                    'original_key': self._batch_key},
                    'summary_stats': {'n_batch': self.n_batch}},
                'labels': {'data_registry': {'attr_name': 'obs',
                    'attr_key': '_scvi_labels'},
                    'state_registry': {'categorical_mapping': self.label_keys,
                    'original_key': self._label_key,
                    'unlabeled_category': 'unlabeled'},
                    'summary_stats': {'n_labels': self.n_labels}},
                'size_factor': {'data_registry': {},
                    'state_registry': {},
                    'summary_stats': {}},
                'extra_categorical_covs': {'data_registry': {},
                    'state_registry': {},
                    'summary_stats': {'n_extra_categorical_covs': 0}},
                'extra_continuous_covs': {'data_registry': {},
                    'state_registry': {},
                    'summary_stats': {'n_extra_continuous_covs': 0}}
            },
            'setup_method_name': 'setup_anndata',
        }

    @property
    def batch_keys(self) -> int:
        if self._batch_key is None:
            return None
        return self._dataset.encoders[self._batch_key]

    @property
    def labels(self) -> np.ndarray:
        if self._label_key is None:
            return None
        return self._dataset.get_merged_labels(self._label_key)

    @property
    def label_keys(self) -> int:
        if self._label_key is None:
            return None
        return self._dataset.encoders[self._label_key]

    def on_before_batch_transfer(
        self,
        batch,
        dataloader_idx,
    ):
        X_KEY: str = "X"
        BATCH_KEY: str = "batch"
        LABEL_KEY: str = "labels"

        labels = batch[self._label_key][:, None]
        labels[labels == -1] = self.n_labels

        return {
            X_KEY: batch["X"].float(),
            BATCH_KEY: batch[self._batch_key][:, None] if self._batch_key is not None else None,
            LABEL_KEY: labels,
        }

    class _InferenceDataloader:
        """Wrapper to apply `on_before_batch_transfer` during iteration."""

        def __init__(self, dataloader, transform_fn):
            self.dataloader = dataloader
            self.transform_fn = transform_fn

        def __iter__(self):
            for batch in self.dataloader:
                yield self.transform_fn(batch, dataloader_idx=None)

        def __len__(self):
            return len(self.dataloader)

In [58]:
collection = ln.Collection.get(name="covid_normal_lung")

In [59]:
datamodule = MappedCollectionDataModule(
    collection,
    batch_key = "assay",
    label_key = "cell_type",
    batch_size = 128,
    join = "inner",
    parallel = True,
)
model = scvi.model.SCVI(adata=None, registry=datamodule.registry)

! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, cal

In [60]:
model.train(max_epochs=1, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/canergen/.local/share/hatch/env/virtual/scvi-tools/SQ6yRUPo/scvi-tools/lib/python3.12/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training:   0%|          | 0/1 [00:00<?, ?it/s]

/home/canergen/Documents/scvi-tools/src/scvi/module/_vae.py:569: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)
/home/canergen/Documents/scvi-tools/src/scvi/module/_vae.py:569: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [61]:
scanvi = scvi.model.SCANVI.from_scvi_model(model, adata=False, unlabeled_category="unlabeled", datamodule=datamodule)

In [62]:
scanvi.train(max_epochs=1, datamodule=datamodule)

INFO     Training for 1 epochs.                                                                                    


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/canergen/.local/share/hatch/env/virtual/scvi-tools/SQ6yRUPo/scvi-tools/lib/python3.12/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training:   0%|          | 0/1 [00:00<?, ?it/s]

/home/canergen/Documents/scvi-tools/src/scvi/module/_scanvae.py:310: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -px.log_prob(x).sum(-1)
/home/canergen/Documents/scvi-tools/src/scvi/module/_scanvae.py:310: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -px.log_prob(x).sum(-1)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [63]:
model.save("lamin_model", save_anndata=False, overwrite=True)

In [64]:
adata = collection.load(join='inner')
#adata.obs["batch"] = adata.obs["assay"]
model2 = model.load_query_data(adata=adata, reference_model="lamin_model")

In [13]:
model.load("lamin_model", adata=False)
model.load_query_data(adata=False, reference_model="lamin_model", registry=datamodule.registry)

INFO     File lamin_model/model.pt already downloaded                                                              
INFO     File lamin_model/model.pt already downloaded                                                              


SCVI model with the following parameters: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, gene_likelihood: zinb, 
latent_distribution: normal.
Training status: Not Trained
Model's adata is minified?: False

In [21]:
model2.train(max_epochs=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/canergen/.local/share/hatch/env/virtual/scvi-tools/SQ6yRUPo/scvi-tools/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training:   0%|          | 0/1 [00:00<?, ?it/s]

/home/canergen/Documents/scvi-tools/src/scvi/module/_vae.py:569: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)
/home/canergen/Documents/scvi-tools/src/scvi/module/_vae.py:569: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [22]:
model3 = model.load("lamin_model", adata=ad)

INFO     File lamin_model/model.pt already downloaded                                                              


TTTTTTTT setup_anndata


/home/canergen/Documents/scvi-tools/src/scvi/data/fields/_base_field.py:63: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [23]:
model3.train(max_epochs=1)
model3.save("lamin_model_anndata", save_anndata=False, overwrite=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/canergen/.local/share/hatch/env/virtual/scvi-tools/SQ6yRUPo/scvi-tools/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training:   0%|          | 0/1 [00:00<?, ?it/s]

/home/canergen/Documents/scvi-tools/src/scvi/module/_vae.py:569: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)
/home/canergen/Documents/scvi-tools/src/scvi/module/_vae.py:569: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)
`Trainer.fit` stopped: `max_epochs=1` reached.


INFO     File lamin_model_anndata/model.pt already downloaded                                                      
INFO     File lamin_model_anndata/model.pt already downloaded                                                      


SCVI model with the following parameters: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, gene_likelihood: zinb, 
latent_distribution: normal.
Training status: Not Trained
Model's adata is minified?: False

In [14]:
dataloader = datamodule.inference_dataloader()

In [15]:
#_ = model.get_elbo(dataloader=dataloader)
#_ = model.get_marginal_ll(dataloader=dataloader)
#_ = model.get_reconstruction_error(dataloader=dataloader)
_ = model.get_latent_representation(dataloader=dataloader)

In [23]:
model.load("lamin_model", adata=ad)

INFO     File lamin_model/model.pt already downloaded                                                              


/home/canergen/Documents/scvi-tools/src/scvi/model/base/_base_model.py:796: UserWarning: `var_names` for the loaded `model` does not match those used to train the model. For valid results, the former should match the latter.
  _validate_var_names(adata, var_names)


ValueError: Please set up your AnnData with SCVI.setup_anndata first.

In [ ]:
model.registry['field_registries']['X']['state_registry']['column_names']

In [45]:
model.get_var_names()

['ENSG00000237491',
 'ENSG00000225880',
 'ENSG00000230368',
 'ENSG00000187634',
 'ENSG00000188976',
 'ENSG00000187961',
 'ENSG00000188290',
 'ENSG00000187608',
 'ENSG00000188157',
 'ENSG00000078808',
 'ENSG00000176022',
 'ENSG00000160087',
 'ENSG00000162572',
 'ENSG00000131584',
 'ENSG00000169972',
 'ENSG00000127054',
 'ENSG00000224051',
 'ENSG00000107404',
 'ENSG00000162576',
 'ENSG00000175756',
 'ENSG00000221978',
 'ENSG00000242485',
 'ENSG00000272455',
 'ENSG00000235098',
 'ENSG00000179403',
 'ENSG00000160072',
 'ENSG00000197785',
 'ENSG00000160075',
 'ENSG00000215014',
 'ENSG00000228594',
 'ENSG00000197530',
 'ENSG00000189409',
 'ENSG00000248333',
 'ENSG00000189339',
 'ENSG00000008128',
 'ENSG00000215790',
 'ENSG00000008130',
 'ENSG00000078369',
 'ENSG00000178821',
 'ENSG00000067606',
 'ENSG00000162585',
 'ENSG00000157933',
 'ENSG00000116151',
 'ENSG00000157916',
 'ENSG00000157911',
 'ENSG00000272449',
 'ENSG00000162591',
 'ENSG00000158109',
 'ENSG00000116213',
 'ENSG00000078900',


In [36]:
model2 = model.load_query_data(adata=ad, reference_model=model)

/home/canergen/Documents/scvi-tools/src/scvi/model/base/_archesmixin.py:118: UserWarning: `var_names` for the loaded `adata` does not match those of the `adata` used to train the model. For valid results, the former should match the latter.
  _validate_var_names(adata, var_names)
/home/canergen/Documents/scvi-tools/src/scvi/model/base/_archesmixin.py:123: UserWarning: `var_names` for the loaded `adata` does not match those of the `adata` used to train the model. For valid results, the former should match the latter.
  _validate_var_names(adata, var_names)


AttributeError: 'DataFrame' object has no attribute 'soma_joinid'

In [34]:
b = model.get_reconstruction_error(adata=ad)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


AttributeError: 'NoneType' object has no attribute 'transfer_fields'

In [29]:
ad.raw.X.data

array([12.,  1.,  8., ...,  1.,  1.,  3.], dtype=float32)

In [33]:
ad.obs

,Celltypes,Loc_true,suspension_type,scsn,donor_id,Gender,Sample,ID,assay_ontology_term_id,Study,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,PoolDon,DonorPool,Protocol_plot,scDonor_snBatch,is_primary_data,disease_ontology_term_id,organism_ontology_term_id,donor_id_2,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,Age range,Smoking status,Years smoking,BMI range,development_stage_ontology_term_id,Location_long,Cell_fraction,tissue_ontology_term_id,cell_type_ontology_term_id,tissue_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,observation_joinid,nCount_RNA,nFeature_RNA,smoking,packyears,percent.mito,batch,dissection,chemistry,percent_mito,n_counts,leiden,phase,S_score,G2M_score,new_celltype,score,log1p_n_genes_by_counts,log1p_total_counts,mito_frac,RBP_frac,treatment,histo,procedure,clusters,author_cell_type,clusters_fine,HTAN_Biospecimen_ID,HTAN_Participant_ID,seurat_clusters,celltype,lineage,nGene,nReads,plate.barcode,cell.id,region,label,sorter,sort.location,sample,location,percent.ercc,percent.ribo,gating,free_annotation,Number of splices: Total,Number of splices: Annotated (sjdb),Number of splices: GT-AG,Number of splices: GC-AG,Number of splices: AT-AC,Number of splices: Non-canonical,"Mapping speed, Million of reads per hour",Average input read length,compartment,artifact_uid
WTDAtest7887999-GGGAGATGTGAGTGAC,B_memory,d_LowLeftPar,cell,cells,A32,M,WTDAtest7887999,A32-LNG-1-SC-45N-1,EFO:0009899,dissociation,655.0,655.0,1676.563354,50.247673,2.997064,nan,nan,LibCD45neg_TrypLibUndigest,A32_cells,False,PATO:0000461,NCBITaxon:9606,411C,PATO:0000384,HANCESTRO:0005,25-30,current,15.0,20-24,HsapDv:0000123,Lower Left Lobe,CD45neg and Liberase undigested,UBERON:0008953,CL:0000787,tissue,memory B cell,10x 3' v2,normal,Homo sapiens,male,lower lobe of left lung,European,29-year-old human stage,sj#d$bsNrL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,qOw3M7KtZ0QjEEtUcSy9
WTDAtest7888001-CTAGAGTCAAGTTCTG,B_plasma_IgG,a_Trachea,cell,cells,A32,M,WTDAtest7888001,A32-LNG-2-SC-45N-1,EFO:0009899,dissociation,1052.0,1052.0,14591.662109,1.973254,0.013523,nan,nan,LibCD45neg_TrypLibUndigest,A32_cells,False,PATO:0000461,NCBITaxon:9606,411C,PATO:0000384,HANCESTRO:0005,25-30,current,15.0,20-24,HsapDv:0000123,Trachea,CD45neg and Liberase undigested,UBERON:0003126,CL:0000985,tissue,IgG plasma cell,10x 3' v2,normal,Homo sapiens,male,trachea,European,29-year-old human stage,(xklu`s@kK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,qOw3M7KtZ0QjEEtUcSy9
WTDAtest7888001-TACTTGTAGGACGAAA,B_plasma_IgG,a_Trachea,cell,cells,A32,M,WTDAtest7888001,A32-LNG-2-SC-45N-1,EFO:0009899,dissociation,1346.0,1346.0,10357.476562,10.057343,0.097102,nan,nan,LibCD45neg_TrypLibUndigest,A32_cells,False,PATO:0000461,NCBITaxon:9606,411C,PATO:0000384,HANCESTRO:0005,25-30,current,15.0,20-24,HsapDv:0000123,Trachea,CD45neg and Liberase undigested,UBERON:0003126,CL:0000985,tissue,IgG plasma cell,10x 3' v2,normal,Homo sapiens,male,trachea,European,29-year-old human stage,PyS2QZSS6l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,qOw3M7KtZ0QjEEtUcSy9
WTDAtest7888002-AAACGGGCAACCGCCA,B_plasma_IgA,a_Trachea,cell,cells,A32,M,WTDAtest7888002,A32-LNG-2-SC-45P-1,EFO:0009899,dissociation,1612.0,1612.0,15639.706055,39.134060,0.250222,nan,nan,LibCD45pos,A32_cells,False,PATO:0000461,NCBITaxon:9606,411C,PATO:0000384,HANCESTRO:0005,25-30,current,15.0,20-24,HsapDv:0000123,Trachea,CD45pos,UBERON:0003126,CL:0000987,tissue,Ig